!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

/tmp/glove.6B.100d.txt

         Your Code here. Create list items where the first item is the text, found in row[5], and the second is the label. 
         Note that the label is a '0' or a '4' in the text. When it's the former, make
         your label to be 0, otherwise 1. Keep a count of the number of sentences in num_sentences



In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Dropout, LSTM, Conv1D, MaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

In [82]:
import numpy as np
import csv
import random

In [21]:
embedding_dim = 100
max_length = 16
padding_type = 'post'
oov_tok = '<OOV>'
training_size=160000
#Your dataset size here. Experiment using smaller values (i.e. 16000), but don't forget to train on at least 160000 to see the best effects
test_portion=.1

In [109]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.8):
            print("\nReached accuracy so cancelling training!")
            self.model.stop_training = True

In [83]:

corpus=[]
with open("/tmp/training_cleaned/training_cleaned.csv") as f:
    reader=csv.reader(f)
    next(reader)
    for row in reader:
        if row[0]=="0":
            label=0
        else:
            label=1
        sentence=row[5]
        corpus.append([label, sentence])


In [84]:
random.shuffle(corpus)

In [86]:
corpus=corpus[:training_size]

In [87]:
len(corpus)

160000

In [88]:
sentences=[sentence for label, sentence in corpus]
labels=[label for label, sentence in corpus]

In [89]:
labels=np.array(labels)

In [90]:
sum(labels)

79739

In [91]:
len(sentences)

160000

In [92]:
tokenizer=Tokenizer()

In [93]:
tokenizer.fit_on_texts(sentences)

In [94]:
seq = tokenizer.texts_to_sequences(sentences)

In [95]:
padded = pad_sequences(seq, maxlen=max_length, padding=padding_type)

In [96]:
padded = np.array(padded)

In [97]:
split_at=int(training_size*test_portion)
X_valid = padded[:split_at]
y_valid = labels[:split_at]

X_train = padded[split_at:training_size]
y_train = labels[split_at:training_size]


In [98]:
vocab_size=len(tokenizer.word_index)

In [99]:
vocab_size

138814

In [100]:
embeddings_index = {};
with open("/tmp/glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word]=coefs

In [101]:
embedding_matrix = np.zeros((vocab_size+1, embedding_dim))

In [102]:
embedding_matrix.shape

(138815, 100)

In [103]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [104]:
for word, i in tokenizer.word_index.items():
    word_vector = embeddings_index.get(word)
    if word_vector is not None:
        embedding_matrix[i]=word_vector

In [105]:
print(len(embedding_matrix))

138815


In [106]:
model = Sequential([
    Embedding(vocab_size+1, embedding_dim, input_length=16, weights=[embedding_matrix], trainable=False),
    Dropout(0.2),
    Conv1D(65, 4, activation='relu'),
    MaxPool1D(4),
    LSTM(64),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 16, 100)           13881500  
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 13, 65)            26065     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 3, 65)             0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33280     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 13,940,910
Trainable params: 59,410
Non-trainable params: 13,881,500
_____________________________________

In [107]:
model.compile(loss='binary_crossentropy', optimizer=Adam(clipvalue=1.0), metrics=['accuracy'])

In [77]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [110]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[myCallback()])

Train on 144000 samples, validate on 16000 samples
Epoch 1/100
144000/144000 [==============================] - 16s 114us/sample - loss: 0.4879 - accuracy: 0.7599 - val_loss: 0.4937 - val_accuracy: 0.7589
Epoch 2/100
144000/144000 [==============================] - 16s 113us/sample - loss: 0.4817 - accuracy: 0.7630 - val_loss: 0.4924 - val_accuracy: 0.7593
Epoch 3/100
144000/144000 [==============================] - 16s 113us/sample - loss: 0.4755 - accuracy: 0.7685 - val_loss: 0.4893 - val_accuracy: 0.7651
Epoch 4/100
144000/144000 [==============================] - 16s 114us/sample - loss: 0.4720 - accuracy: 0.7702 - val_loss: 0.4886 - val_accuracy: 0.7641
Epoch 5/100
144000/144000 [==============================] - 16s 114us/sample - loss: 0.4672 - accuracy: 0.7732 - val_loss: 0.4912 - val_accuracy: 0.7610
Epoch 6/100
144000/144000 [==============================] - 16s 114us/sample - loss: 0.4622 - accuracy: 0.7785 - val_loss: 0.4879 - val_accuracy: 0.7625
Epoch 7/100
144000/144000

144000/144000 [==============================] - 17s 115us/sample - loss: 0.4248 - accuracy: 0.7998 - val_loss: 0.5090 - val_accuracy: 0.7575
Epoch 54/100
144000/144000 [==============================] - 16s 112us/sample - loss: 0.4255 - accuracy: 0.7982 - val_loss: 0.5086 - val_accuracy: 0.7583
Epoch 55/100
144000/144000 [==============================] - 16s 109us/sample - loss: 0.4244 - accuracy: 0.7994 - val_loss: 0.5116 - val_accuracy: 0.7599
Epoch 56/100
143744/144000 [============================>.] - ETA: 0s - loss: 0.4248 - accuracy: 0.8008
Reached accuracy so cancelling training!
144000/144000 [==============================] - 17s 117us/sample - loss: 0.4248 - accuracy: 0.8008 - val_loss: 0.5072 - val_accuracy: 0.7592
